# Recall on POMDPs

### Recall: the POMDP model

Let's recall that a POMDP (Partially Observable Markov Decision Process) is a tuple $\langle S, A, \Omega, p(), o(), r(), b_0 \rangle$, with $S$ (resp. $A$, $\Omega$) the state (resp. action, observation) space.
How does this probabilistic planning model work?
1. At each time step $t \in \mathbb{N}$, the artificial agent has only a partial knowledge of the actual current state $s \in S$. In order to still be able to reason about the state of the system, the agent maintains a probability distribution over $S$, called belief state, and denoted by $b_t$. The initial belief state is $b_0$ and takes part in the definition of the POMDP as a prior distribution about the initial state.
2. When the artificial agent selects an action $a \in A$, the probability that the state of the system switches from $s \in S$ to $s' \in S$, is $Pr(s'\mid s,a) = p(s,a,s')$ given by the transition probability function $p()$ in the POMDP definition.
3. The current state is not available to the artificial agent, but the latter still receives information given by the observation $o' \in \Omega$. The probability that the observation $o' \in \Omega$ is received, is $Pr(o'\mid s',a) = o(a,s',o')$ given by the observation probability function $o()$. 
4. An action $a$ applied in a state $s$ is rewarded by $r(s,a) \in \mathbb{R}$ given by the reward function $r()$.

![POMDP_for_HRI.png](figures/POMDP_for_HRI.png)


#### The belief state:
1. In partially observable domains, only successive actions and observations are visible to the agent to implement the state estimation. These sequences can be seen as the agent's memory.
2. Let's define the history of all observations received and actions performed at time step $t \in \mathbb{N}$: $h_t := \{a_0, o_1,...,o_{t-1},a_{t-1},o_t\}$. The size of the history grows with the time step $t \in \mathbb{N}$, so it can become huge very quickly.
3. The definition of the belief state at time step $t \in \mathbb{N}$ is the probability of the current state $s_t \in S$ given the history $h_t$: $b_t(s) = Pr(s_t = s \mid h_t)$.

However, in practice, it is not necessary to keep the history in memory to compute the belief state. It is indeed possible to simply update the previous belief state using the action performed, and the observation received:
\begin{eqnarray*}
{b_{t+1}(s')} & = & Pr(s' \mid h_{t+1}) = Pr(s' \mid  h_t, a_t, o_{t+1})
 = \dfrac{Pr(o_{t+1},s' \mid h_t,a_t)}{Pr(o_{t+1} \mid h_t,a_t)} \nonumber \\
& = & \dfrac{Pr(o_{t+1} \mid s', a_t) Pr(s' \mid h_t,a_t) }{Pr(o_{t+1} \mid h_t, a_t)} 
= \dfrac{o(a_t, s', o_{t+1}) Pr(s' \mid h_t,a_t) }{Pr(o_{t+1} \mid h_t,a_t)} \nonumber \\
& = & \dfrac{ o(a_t, s', o_{t+1}) \sum_{s \in S} Pr(s' \mid s, a_t) Pr(s \mid h_t) }{ Pr(o_{t+1} \mid h_t,a_t)} 
= \dfrac{ o(a_t, s', o_{t+1}) \sum_{s \in S}p(s,a_t,s') b_t(s) }{ Pr(o_{t+1} \mid h_t,a_t)} \nonumber \\
& = & \dfrac{ o(a_t, s', o_{t+1}) \sum_{s \in S}p(s,a_t,s') b_t(s) }{ \sum_{(s,s') \in S^2} o(a_t, s', o_{t+1}) p(s,a_t,s') b_t(s) } \nonumber = U(b_t,a_t,o_{t+1})(s').
%{b_o^a(s')} & = & p(s' \mid b,a,o) = \dfrac{p(o,s' \mid b,a)}{p(o \mid b,a)} \nonumber \\
%& = &\dfrac{p(o \mid s',a)p(s' \mid b,a)}{p(o \mid b,a)}\nonumber \\
%& = &\dfrac{p(o \mid s',a)p(s' \mid b,a)}{\sum_{s,s'}p(o \mid b,a,s,s')p(s,s'\mid b,a)}\nonumber\\
%& = &\dfrac{p(o \mid s',a)\sum_{s\in S} p(s' \mid s,a)b(s)
%}{\sum_{s'\in S}p(o \mid s',a)\sum_{s\in S} p(s'\mid s,a)b(s)}
\label{beliefupdate}
\end{eqnarray*}
The belief update function $U$ can be defined using transition and observation functions $p()$ and $o()$. When the artificial agent uses action $a_t$ and receives observation $o_{t+1}$, it can use $b_{t+1} = U(a_t,o_{t+1},b_t)$ as belief on the system state $s_{t+1}$. Now, if the belief state is $b_t$, and the selected action is $a_t$, the probability of obtaining the belief state $b_{t+1} = U(b_t,a_t,o_{t+1})$ is the following:
\begin{eqnarray*}
Pr(o_{t+1} \mid  h_t, a_t) & = & \sum_{s' \in S} Pr(o_{t+1} \mid s', a_t) Pr(s' \mid h_t) = \sum_{(s,s') \in S^2} Pr(o_{t+1} \mid s', a_t) Pr(s' \mid s, a_t) Pr(s \mid h_t)\\ 
& = & \sum_{(s,s') \in S^2} o(a_t, s', o') p(s, a_t, s') b_t(s).
\end{eqnarray*}
Since the probability distribution on the next belief $b_{t+1}$ only depends on the previous belief $b_t$ and the current action $a_t$, the belief state process $(b_t)_{t \in \mathbb{N}}$ is a Markov Decision Process (MDP) whose system space is the space of probability distributions. As a result, a POMDP can be transformed into a belief state MDP.

In short, the artificial agent synthesizes all available information from the past using a belief state $b_t(s)$, which is a probability distribution on the system state updated every time step: $b_t(s) := Pr(s_t=s|o_t,a_{t-1},o_{t-1},..., a_0) = U(a_{t-1},o_t,b_{t-1})(s)$.

#### POMDP solving objective

The agent's objective is to choose actions that will drive him to achieve better rewards. Let's define a deterministic Markovian policy $\pi(b)$ as a function such as, $\pi (b): b\mapsto a$. The expected value of a given action policy $\pi (b)$ can be formilized as:

$V^{\pi}(b) = E_\pi\left[\sum ^{\infty} _{t=0} \gamma ^{t} r(b_t,\pi(b_t)) \middle\vert b_0=b\right]$

where,  $0 \leqslant \gamma < 1$, and $r(b_t,\pi(b_t))=\sum_{s\in S}r(s,\pi(b_t))b_t(s)$

In the POMDP context, we search for the optimal policy $\pi^*(b)$ that maximizes the expectation of *the sum of discounted rewards* ($\gamma$-discounted criteron). 

$V^{\pi^*}(b) = \max_\pi E_\pi\left[\sum ^{\infty} _{t=0} \gamma ^{t} r(b_t,\pi(b_t)) \middle\vert b_0=b\right]$

Opening the sum above, on can retrieve the Bellman equation:

$V^{\pi^*}(b)  =  \max_{a\in A } \left[ \sum_{s\in S}r(s,a)b(s) + \gamma\sum_{o \in \Omega} p(o|a,b)V^{\pi^*}(b^a_o)\right]$

Thus, an optimal policy $\pi^*$ is defined by $V^{\pi^*}$ that satisfies this Bellman. The solution is said optimal when $V^*=V^{\pi^*}$ converges to a fixed point for any $b$. In other words, one could iterate on this value function $\forall b$ until it converges, and then extract the related policy.

However:
1. the exact solution in infinite horizon settings is *hard* to compute (PSPACE-complete)
2. actually, current algorithms approach the optimal solution, exploiting particular properties: of the value function and of the system dynamics.

#### Value Function parametrization

1. The infinite horizon value function $V^{\pi}(b) = E_{\pi}\left[\sum ^{\infty} _{t=0} \gamma ^{t} r(b_t,\pi(b_t)) \middle\vert b_0=b\right]$ can be approximated by the finite horizon optimal value function $V^{\pi}(b) = E_{\pi}\left[\sum ^{N} _{t=0} r(b_t,\pi(b_t)) \middle\vert b_0=b\right]$ which is a piece-wise linear and convex (PWLC) function.

2. If the function $F: \mathbb{R}^{S} \rightarrow \mathbb{R}$ is a PWLC function,
it exists a finite set of vectors of $\mathbb{R}^{S}$,
denoted by $\{ \alpha_i \}_{i=1}^n$, such that $F$ can be written 
$F(b) = \displaystyle \max_{i=1}^n \alpha_i \cdot b = \max_{i=1}^n \sum_{s \in S} \alpha_{i}(s) \cdot b(s)$.
Since the belief state $b$ is a probability distribution over $S$, 
we can write $b: S \rightarrow [0,1]$, and even $b \in \mathbb{R}^S$. 
The approximation of the optimal value function 
$V: \mathbb{R}^S \rightarrow \mathbb{R}$ 
can be parameterized by a set of $\alpha$-vectors
$\Gamma_n = \left\lbrace \alpha_i \right\rbrace_{i=1}^n$, with $\alpha_i \in \mathbb{R}^S$, $\forall i$.
Indeed,
$V(b) = \displaystyle \max_{\alpha \in \Gamma_n} \alpha \cdot b = \max_{\alpha \in \Gamma_n} \sum_{s \in S} \alpha(s) \cdot b(s)$.
$%i \in 1,...,|V_n|$
3. In fact, $V$ is defined on the belief space
$\{ b \in \mathbb{R}^S \mid \sum_s b(s)=1 \}$.
Each $\alpha$-vector $\alpha_i \in \mathbb{R}^S$
is associated with an action $a(\alpha) \in A$,
and defines a region of the belief space, 
where $\alpha_i \cdot b = \max_j \alpha_j \cdot b = V(b)$,
that is, where $\alpha_i \cdot b$ reaches is maximum.
The $\alpha$-vectors form then a partition of the belief state.

$$%V_n(b)  =  \max _{\alpha ^i_n \in \Gamma_n} \sum _{s \in S} b(s)\alpha ^i_n(s) \rightarrow V_n(b)  =  \max_{\alpha ^i_n \in \Gamma_n}b\cdot \alpha _n^i$$ 

Thus, for a given belief state $b$, the gradient of the value function is induced by the vector:

$$ \alpha^b = \operatorname*{arg\,max}_{\alpha \in \Gamma_n} b \cdot \alpha $$

and, the associated policy $\pi(b) \in A$ is the action of this $\alpha$-vector, that is: 
$\pi(b) = a(\alpha^b)$. 

#### Solving Algorithm : point-based value iteration  (PBVI)

#### Value backup operation:
Let's define a $V_n$ parameterized by a set $\Gamma_n$ of $\alpha$-vectors and a given belief state $b$. One can compute $\alpha^b_{n+1}$ of $V_{n+1}$ from $LV_n$ (the unknown set of vectors):
$\alpha _{n+1}^b = arg\max_{\alpha _{n+1}^i \in \mathcal{L}V_n} b \cdot \alpha _{n+1}^i,$

Let's define $r_a=r(s,a)$ and $b \cdot r_a = \sum_s b(s)r(s,a)$, and a set of belief states $\mathcal B$ where $b\in \mathcal B$, we have:
$V_{n+1}(b) = \max _a \left[ b \cdot r_a + \gamma \sum _o p(o \mid a,b)V_n(b_o^a) \right]$

$ = \max _a \left[ b \cdot r_a + \gamma \sum _o p(o \mid b,a) \max _{\alpha _{n}^i \in V_n} \sum _{s'}b_o^a(s')\alpha _n^i(s')\right]$

$ =  \max _a \left[ b \cdot r_a + \gamma \sum _o p(o\mid b,a) \max _{\alpha _{n}^i \in V_n} \frac {\sum _{s'} p(o \mid s',a) \sum _s p(s' \mid s,a)b(s)\alpha _n^i(s')}{p(o\mid b,a)}\right]$

$ =  \max _a \left[ b \cdot r_a + \gamma \sum _o \max _{\alpha _{n}^i \in V_n} \sum _s \sum _{s'} p(o \mid s',a) p(s' \mid s,a)b(s)\alpha _n^i(s')\right]$

thus: $ V_{n+1}(b)  =  \max _a \left[ b \cdot r_a + \gamma \sum _o \max _{\alpha^{a,o}_i \in \Gamma^{a,o}} b \cdot \alpha^{a,o}_i \right] $

Then, applying $\max _j b \cdot \alpha _j = b \cdot \operatorname*{arg\,max}_j b\cdot \alpha_j$ two times, we have:

$backup(b)  =  arg\max_{\alpha^{a}_b \in \Gamma^a_b} b \cdot \alpha^a_b$

$\mathrm{with} \; \Gamma^a_b  \leftarrow  r_a + \gamma \sum _o arg\max_{\alpha^{a,o}_i \in \Gamma^{a,o}} b \cdot \alpha^{a,o}_i$

#### Algorithm

<div>
<img src="figures/pbvi.png" width="600"/>
</div>


The output of this algorithm is a set of $\alpha$-vectors $\Gamma$ approximating the value function. Each $\alpha$-vector has an associated action $a$.

At execution time, one can determine the best action for a given belief state $b$, appying:

$$ \alpha^b = \operatorname*{arg\,max}_{\alpha \in \Gamma} b \cdot \alpha $$

and, the associated policy $\pi(b) \in A$ is the action of this $\alpha$-vector, that is $\pi(b) = a(\alpha^b)$. 


#### Provided library
The PyPOMDP library (see https://github.com/namoshizun/PyPOMDP) is provided with this notebook. 

This library proposes two state-of-the-art algorithms, PBVI and POMCP (do not hesitate to get a look in the Caroline's slides courses). However, this library has been customized with several additional features that will be very useful for this practical work.

For instance, the library now save a Value Function parametrized by $\alpha$-vectors, in a file. This can be then reloaded to simulate the policy in order to evaluate futur gains, or in order to replay an experiment.

In this practical work course we will mainly use the implementation of PBVI.